<a href="https://colab.research.google.com/github/RiccardoRubini93/SWA_DQ/blob/main/create_9899_big_query_test_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [2]:
project_id = "advanced-analytics-278408"
from google.cloud import bigquery
client = bigquery.Client(project=project_id)

In [3]:
#define array on which we need to loop

query_schema_list = '''
        SELECT
        ARRAY_AGG(CONCAT(project_id,".", 	dataset_id,".",table_id ))  
        FROM
        advanced-analytics-278408.data__1st_layer.9891__monitoring_information_and_params
        WHERE is_to_be_monitored= TRUE
        '''

schema_list = client.query(query_schema_list).to_dataframe()
schema_list.head()

,f0_
0,[advanced-analytics-278408.data__1st_layer.989...


In [4]:
query_date_loop = '''
        SELECT
        ARRAY_AGG(data_loop_field)  
        FROM
        advanced-analytics-278408.data__1st_layer.9891__monitoring_information_and_params
        WHERE is_to_be_monitored= TRUE
        '''

date_loop_list = client.query(query_date_loop).to_dataframe()
date_loop_list.head()

,f0_
0,"[analysis_date, order_date, DATE, DATE, DATE, ..."


In [5]:
query_id_loop = '''
        SELECT
        ARRAY_AGG(id_field)  
        FROM
        advanced-analytics-278408.data__1st_layer.9891__monitoring_information_and_params
        WHERE is_to_be_monitored= TRUE
        '''

id_loop_list = client.query(query_id_loop).to_dataframe()
id_loop_list.head()

,f0_
0,"[analysis_date_time, transaction_id, DATE, DAT..."


'transaction_id'

In [11]:
#create the initial table where to append the new data to

from google.cloud.exceptions import NotFound

table_id = "advanced-analytics-278408.user__rubini_riccardo.9899_big_query_test_v3"

try:
  client.get_table(table_id)  # Make an API request.
  print("Table {} already exists.".format(table_id))

except NotFound:
  print("Table {} is not found, creating it....".format(table_id))
  query_create_statement = f'''
              create table {table_id} as (
                select t1.*, t2.* FROM (with temp_tb as (select 'advanced-analytics-278408.data__1st_layer.9899__pipelines_target_tables_metadata' as full_table_id ,CAST(analysis_date as STRING) as order_date, count(distinct(analysis_date_time)) as transaction_id_cnt,count(*) as cnt FROM advanced-analytics-278408.data__1st_layer.9899__pipelines_target_tables_metadata group by analysis_date order by order_date desc ), aggregated_table as ( select full_table_id, array_agg(struct(order_date,transaction_id_cnt,cnt)) as day_list FROM temp_tb group by full_table_id), master_tb as (SELECT 'advanced-analytics-278408.data__1st_layer.9899__pipelines_target_tables_metadata' as full_table_id , count(*) as total_rows, CAST(min(analysis_date) AS STRING) as min_date, CAST(max(analysis_date) AS STRING) as max_date, count(distinct analysis_date_time) as id_distinct_count FROM advanced-analytics-278408.data__1st_layer.9899__pipelines_target_tables_metadata) select t1.full_table_id,t2.min_date,t2.max_date,t2.id_distinct_count,t2.total_rows,t1.day_list FROM aggregated_table t1 join master_tb t2 on (t1.full_table_id = t2.full_table_id)) AS t1 JOIN `advanced-analytics-278408.data__1st_layer.9891__monitoring_information_and_params` as t2 on t1.full_table_id = t2.complete_table_id
              ); 
  '''
  create_table_9899 = client.query(query_create_statement)
  rows = create_table_9899.result()


Table advanced-analytics-278408.user__rubini_riccardo.9899_big_query_test_v3 is not found, creating it....


In [ ]:
print(schema_list.values[0][0][1])
print(date_loop_list.values[0][0][1])
print(id_loop_list.values[0][0][1])

advanced-analytics-278408.data__1st_layer.0005__customer_transactions_online_eur
order_date
transaction_id


In [12]:
#loop over the lists previously defined to append the new columns

#N = schema_list.values[0][0].shape[0]
history_limit = 50

N = 5

for i in range(1,N):

  print(f"Step : {i} -- Writing info for : {schema_list.values[0][0][i]}" )

  string = f'''
          select t1.*, t2.* FROM (with temp_tb as (select '{schema_list.values[0][0][i]}' as full_table_id ,
          CAST({date_loop_list.values[0][0][i]} as STRING) as order_date, 
          count(distinct({id_loop_list.values[0][0][i]})) as transaction_id_cnt,
          count(*) as cnt FROM {schema_list.values[0][0][i]} group by {date_loop_list.values[0][0][i]} order by {date_loop_list.values[0][0][i]} desc limit {history_limit}), 
          aggregated_table as ( select  full_table_id, array_agg(struct(order_date,transaction_id_cnt,cnt)) as day_list FROM  temp_tb  group by full_table_id), 
          master_tb as (SELECT '{schema_list.values[0][0][i]}' as full_table_id , count(*) as total_rows, CAST(min({date_loop_list.values[0][0][i]}) AS STRING) as min_date, 
          CAST(max({date_loop_list.values[0][0][i]}) AS STRING) as max_date, count(distinct {id_loop_list.values[0][0][i]}) as id_distinct_count 
          FROM {schema_list.values[0][0][i]}) select t1.full_table_id,t2.min_date,t2.max_date,t2.id_distinct_count,t2.total_rows,t1.day_list 
          FROM aggregated_table t1 join master_tb t2 on (t1.full_table_id = t2.full_table_id)) AS t1 
          JOIN  `advanced-analytics-278408.data__1st_layer.9891__monitoring_information_and_params` as t2 
          on t1.full_table_id = t2.complete_table_id;
          '''
  insert_string = 'insert into `advanced-analytics-278408.user__rubini_riccardo.9899_big_query_test_v3`' + string

  insert_test_job = client.query(insert_string)
  rows = insert_test_job.result()
  
    

Step : 1 -- Writing info for : advanced-analytics-278408.data__1st_layer.0005__customer_transactions_online_eur
Step : 2 -- Writing info for : advanced-analytics-278408.Draft_Data_Model.0004__v3__traffic_transaction_adj
Step : 3 -- Writing info for : advanced-analytics-278408.Draft_Data_Model.0006__cogs
Step : 4 -- Writing info for : advanced-analytics-278408.Draft_Data_Model.0019__cogs_csl


In [14]:
#cells that compute number of fields with description and a object with list of fields for each table

query_info_extra = '''
    create or replace table `advanced-analytics-278408.user__rubini_riccardo.9899_with_table_info` as (

    with tb_9899_column_info as (
      SELECT 
        concat(t1.table_catalog,'.',t1.table_schema,'.',t1.table_name) as full_table_id , array_agg(struct(t2.ordinal_position,t1.column_name,t1.data_type,t1.description)) as fields
        FROM `advanced-analytics-278408.data__1st_layer.INFORMATION_SCHEMA.COLUMN_FIELD_PATHS` as t1
        INNER JOIN 
        advanced-analytics-278408.data__1st_layer.INFORMATION_SCHEMA.COLUMNS as t2
        ON (concat(t1.table_catalog,'.',t1.table_schema,'.',t1.table_name) = concat(t2.table_catalog,'.',t2.table_schema,'.',t2.table_name))
        AND (t1.column_name = t2.column_name)
        group by concat(t1.table_catalog,'.',t1.table_schema,'.',t1.table_name)
        
        union all 

        SELECT 
        concat(t1.table_catalog,'.',t1.table_schema,'.',t1.table_name) as full_table_id , array_agg(struct(t2.ordinal_position,t1.column_name,t1.data_type,t1.description)) as fields
        FROM `advanced-analytics-278408.Draft_Data_Model.INFORMATION_SCHEMA.COLUMN_FIELD_PATHS` as t1
        INNER JOIN 
        advanced-analytics-278408.Draft_Data_Model.INFORMATION_SCHEMA.COLUMNS as t2
        ON (concat(t1.table_catalog,'.',t1.table_schema,'.',t1.table_name) = concat(t2.table_catalog,'.',t2.table_schema,'.',t2.table_name))
        AND (t1.column_name = t2.column_name)
        group by concat(t1.table_catalog,'.',t1.table_schema,'.',t1.table_name)

    ),

    tb_field_with_description as (
    select 
              concat(table_catalog,'.',table_schema,'.',table_name) as full_table_id, 
              sum(case when description is not null then 1 else 0 end) as described_fields_count 
              from advanced-analytics-278408.data__1st_layer.INFORMATION_SCHEMA.COLUMN_FIELD_PATHS
              group by concat(table_catalog,'.',table_schema,'.',table_name)
              union all 
              select concat(table_catalog,'.',table_schema,'.',table_name) as full_table_id, 
              sum(case when description is not null then 1 else 0 end) as described_fields_count 
              from advanced-analytics-278408.Draft_Data_Model.INFORMATION_SCHEMA.COLUMN_FIELD_PATHS
              group by concat(table_catalog,'.',table_schema,'.',table_name)
    ),

    tb_info_label as (
      select 
      concat(table_catalog,'.',table_schema,'.',table_name) as full_table_id,
      --array_agg(struct(split(trim(option_value,'[]'),'STRUCT'))) as labels
      split(trim(option_value,'[]'),'STRUCT') as labels
      from data__1st_layer.INFORMATION_SCHEMA.TABLE_OPTIONS
      where option_name = 'labels'
      --group by concat(table_catalog,'.',table_schema,'.',table_name)

      union all

      select 
      concat(table_catalog,'.',table_schema,'.',table_name) as full_table_id,
      --array_agg(struct(split(trim(option_value,'[]'),'STRUCT'))) as labels
      split(trim(option_value,'[]'),'STRUCT') as labels
      from Draft_Data_Model.INFORMATION_SCHEMA.TABLE_OPTIONS
      where option_name = 'labels'
      --group by concat(table_catalog,'.',table_schema,'.',table_name)
    )

    select t1.*,t2.fields,t3.described_fields_count,t4.labels
    from advanced-analytics-278408.user__rubini_riccardo.9899_big_query_test_v3 as t1
    inner join tb_9899_column_info as t2
    on (t1.full_table_id = t2.full_table_id)
    inner join tb_field_with_description as t3
    on (t1.full_table_id = t3.full_table_id)
    left join tb_info_label as t4
    on (t1.full_table_id = t4.full_table_id)
    );

'''
job = client.query(query_info_extra)
res = job.result()


In [ ]:
#join the newly created table with the metadata in the master 
#insert table into advanced-analytics-278408.data__1st_layer.9899__pipelines_target_tables_metadata

final_table_id = 'advanced-analytics-278408.Draft_Data_Model.9899__pipelines_target_tables_metadata_v2'

#define prefix to distinguish whether the table exist already or not

insert = f'insert into {final_table_id}'
create = f'create table {final_table_id} as  ('

#define query to be run to populate the table

sql_body = '''
    select tc1.*, tc2.ddl, tc2.table_ddl_extracted_description

FROM 

(select  t_meta_table.*, min_date, max_date, 	
--id_distinct_count, 
t_meta_topic.day_list,
t_meta_topic.fields,
t_meta_topic.described_fields_count,
complete_table_id, table_code, is_to_be_monitored, current_development_status, current_development_status_id, source_system_id, source_system_name, source_system_ownership_type, extraction_tool, extraction_script, extraction_is_scheduled, extraction_schedule_pattern, landing_zone, landing_zone_path, landing_zone_form, loading_tool, load_script, load_is_scheduled, load_schedule_pattern, load_form, is_dx_framework_applied, id_field, data_loop_field, data_loop_granularity, overall_documentation_URL, to_do_pending, notes 

from

(SELECT CURRENT_DATE() AS analysis_date, CURRENT_DATETIME()AS analysis_date_time, project_id,   dataset_id,   table_id,   TIMESTAMP_MILLIS(creation_time) AS creation_datetime, TIMESTAMP_MILLIS(last_modified_time) AS last_modified_datetime,  column_count, row_count,      size_bytes, ROUND((size_bytes)/pow(10,6),2) as size_megabytes, ROUND((size_bytes)/pow(10,9),2) as size_gigabytes  , type AS table_type

FROM (Select * FROM advanced-analytics-278408.Draft_Data_Model.__TABLES__ UNION ALL Select * FROM advanced-analytics-278408.data__1st_layer.__TABLES__) as t1

JOIN (SELECT table_catalog,     table_schema,   table_name, max(ordinal_position) as column_count
FROM  (Select * FROM `advanced-analytics-278408.Draft_Data_Model.INFORMATION_SCHEMA.COLUMNS` UNION ALL Select * FROM `advanced-analytics-278408.data__1st_layer.INFORMATION_SCHEMA.COLUMNS`) 

group by table_catalog,     table_schema,   table_name) as t2

on t1.table_id = t2.table_name) as t_meta_table


JOIN (SELECT * from advanced-analytics-278408.user__rubini_riccardo.9899_with_table_info ) as t_meta_topic

on CONCAT(t_meta_table.project_id,".", 	t_meta_table.dataset_id,".",t_meta_table.table_id) = t_meta_topic.full_table_id


order by table_id) as tc1 LEFT JOIN


(SELECT CONCAT(table_catalog,".", table_schema,".", table_name) as unique_table_id, creation_time, ddl, CASE WHEN ddl like CONCAT("%)","","OPTIONS(","","  description=%") THEN
SPLIT(ddl, CONCAT(")","","OPTIONS(","","  description="))[SAFE_OFFSET(ARRAY_LENGTH(SPLIT(ddl, CONCAT(")","","OPTIONS(","","  description="))) - 1)]
  ELSE "Description not yet available"
  END AS table_ddl_extracted_description FROM `advanced-analytics-278408.data__1st_layer.INFORMATION_SCHEMA.TABLES`

  union all
  
SELECT CONCAT(table_catalog,".", table_schema,".", table_name) as unique_table_id, creation_time, ddl, CASE WHEN ddl like CONCAT("%)","","OPTIONS(","","  description=%") THEN
SPLIT(ddl, CONCAT(")","","OPTIONS(","","  description="))[SAFE_OFFSET(ARRAY_LENGTH(SPLIT(ddl, CONCAT(")","","OPTIONS(","","  description="))) - 1)]
  ELSE "Description not yet available"
  END AS table_ddl_extracted_description FROM `advanced-analytics-278408.Draft_Data_Model.INFORMATION_SCHEMA.TABLES`) as tc2
  
  ON tc1.complete_table_id = tc2.unique_table_id    
'''

#check if table exsist, if not create one and insert data in the new table

try:
  client.get_table(final_table_id)  # Make an API request.
  print("Table {} already exists, insert new rows....".format(final_table_id))

  #build query_string
  query_string = insert + sql_body + ';'

  job = client.query(query_string)
  res = job.result()

except NotFound:
  print("Table {} is not found, creating it....".format(final_table_id))
  
  #build query_string
  query_string = create + sql_body + ');'

  job = client.query(query_string)
  res = job.result()



Table advanced-analytics-278408.Draft_Data_Model.9899__pipelines_target_tables_metadata_v2 already exists, insert new rows....
